In [ ]:
# 9-1
!mkdir chap9
%cd ./chap9

In [2]:
# 9-3
import random
from tqdm import tqdm
import unicodedata

import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import pytorch_lightning as pl

# 日本語の事前学習済みモデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [3]:
# 9-4
# =============================================================================
# SC_tokenizer（誤変換補正用トークナイザ）
# -----------------------------------------------------------------------------
# 目的（理論）：
# - 「誤変換を含む文」を入力 x とし、「正しい文」をラベル y（語彙上のトークンID列）として
#   学習することで、**各位置ごとに“正しいトークンID”を分類**するタスクへ還元する。
# - これは BERT をエンコーダとし、出力ヘッドが **語彙サイズ |V|** のクラス分類を
#   各タイムステップに対して行う設計（トークン分類）に相当する。
#   * 損失関数の典型：位置 t ごとに CrossEntropyLoss（教師は正解トークンID）。
#   * BERT の “Masked LM” と違い、ここでは**全面的に教師強制**で全位置を監督できる（PAD を除外するなら ignore_index を活用）。
# - この形式は **置換型の誤り**（打鍵ミス・漢字/かな揺れ・全半角揺れ等）に強いが、
#   **挿入/削除**や**語順入れ替え**などの編集距離を伴う変化は「位置合わせ」の前提を崩すため苦手。
#   必要に応じてアライメント戦略（例：差分アルゴリズムでの整列→同一長に拡張）やシーケンス生成系（seq2seq）を検討する。
# - 推論時は各位置の予測トークンIDから文字列を復号し、元文の空白を保持しつつ**スパン単位で置換**して復元する。
#   サブワード（WordPiece）の '##' 接頭辞は接続時に除去する。
#
# 実装上の注意：
# - encode_plus_tagged では、正解文の input_ids をそのまま labels に格納する。
#   学習ループ側で特殊トークンや PAD を損失に含めない場合、labels の該当位置を **-100** に置換しておく（ignore_index）。
# - encode_plus_untagged では、WordPiece の各サブワードを元文字列の部分スパンにマッピングする。
#   繰り返し部分や空白が多い場合、現在の **前方貪欲マッチ**は誤対応のリスクがある（必要なら正規表現や LCS によるロバスト化を検討）。
# - convert_bert_output_to_text では NFKC 正規化を適用している。
#   学習・推論で正規化を**一貫**させないと、スパンの境界と生成表記の齟齬を招く可能性がある。
# =============================================================================


class SC_tokenizer(BertJapaneseTokenizer):

    def encode_plus_tagged(self, wrong_text, correct_text, max_length=128):
        """
        ファインチューニング時に使用。
        誤変換を含む文章と正しい文章を入力とし、
        符号化を行いBERTに入力できる形式にする。

        理論メモ：
        - 入力 x = wrong_text のトークン列に対し、教師 y = correct_text のトークンID列を
          そのまま labels として与えることで、各位置の正解トークンを分類するタスクにする。
        - モデル側は「トークン分類ヘッド（出力次元＝語彙サイズ |V|）」を持ち、
          CrossEntropyLoss(logits_t, label_id_t) を位置 t で計算し、時系列平均を取るのが一般的。
        - PAD/CLS/SEP を損失から除外したい場合、labels の該当位置を -100 にする（ignore_index）。
        """
        # 誤変換した文章をトークン化し、符号化
        encoding = self(
            wrong_text, max_length=max_length, padding="max_length", truncation=True
        )
        # 正しい文章をトークン化し、符号化
        encoding_correct = self(
            correct_text, max_length=max_length, padding="max_length", truncation=True
        )
        # 正しい文章の符号をラベルとする
        # 注意：このままだと [CLS]/[SEP]/[PAD] も学習対象に含まれる。
        # もし PAD を損失から除きたいなら、学習側で labels を -100 に置換する処理を追加する。
        encoding["labels"] = encoding_correct["input_ids"]

        return encoding

    def encode_plus_untagged(self, text, max_length=None, return_tensors=None):
        """
        文章を符号化し、それぞれのトークンの文章中の位置も特定しておく。

        理論メモ：
        - 後段の復号（convert_bert_output_to_text）では、各サブワードが
          原文テキストのどのスパンに対応するかが必要になる。
        - ここでは形態素（MeCab）→サブワード（WordPiece）→原文スパンという対応を
          前方探索で求めている。繰り返しや空白が多い場合は曖昧性が残る点に注意。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = []  # トークンを追加していく。
        tokens_original = []  # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text)  # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)
            if tokens_word[0] == "[UNK]":  # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend(
                    [token.replace("##", "") for token in tokens_word]
                )

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        # アルゴリズム：前方貪欲一致（最短一致ではなく、順次スライド）
        # 注意：同一サブ文字列が繰り返される場合に誤対応のリスクがある。
        position = 0
        spans = []  # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position : position + l]:
                    position += 1
                else:
                    spans.append([position, position + l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        # prepare_for_model が [CLS]/[SEP] の付与と長さ正規化（padding/truncation）を行う。
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding="max_length" if max_length else False,
            truncation=True if max_length else False,
        )
        sequence_length = len(encoding["input_ids"])
        # 特殊トークン[CLS]に対するダミーのspanを追加（-1 は「実体なし」の番兵）。
        spans = [[-1, -1]] + spans[: sequence_length - 2]
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == "pt":
            encoding = {k: torch.tensor([v]) for k, v in encoding.items()}

        return encoding, spans

    def convert_bert_output_to_text(self, text, labels, spans):
        """
        推論時に使用。
        文章と、各トークンのラベルの予測値、文章中での位置を入力とする。
        そこから、BERTによって予測された文章に変換。

        理論メモ：
        - labels は「各位置の語彙ID（予測トークン）」と想定。すなわちモデル出力は
          各位置で |V| 分類を行うトークン分類ヘッドの argmax 等。
        - サブワード '##' は接続時に除去し、文字列として連結。
        - 本実装は NFKC 正規化を通すため、**学習・評価で NFKC を統一**しないと
          スパン境界と生成表記のずれが起き得る（評価の完全一致が過小になる可能性）。
        """
        assert len(spans) == len(labels)

        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # BERTが予測した文章を作成
        # 原文の空白や未対称な部分は、スパンの「穴」をそのまま複製して保持。
        predicted_text = ""
        position = 0
        for label, span in zip(labels, spans):
            start, end = span
            if position != start:  # 空白の処理（原文の未対応部分をそのままコピー）
                predicted_text += text[position:start]
            predicted_token = self.convert_ids_to_tokens(label)
            predicted_token = predicted_token.replace("##", "")
            predicted_token = unicodedata.normalize("NFKC", predicted_token)
            predicted_text += predicted_token
            position = end

        return predicted_text

In [4]:
# 9-5
tokenizer = SC_tokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


In [5]:
# 9-6
# =============================================================================
# 目的（理論）：
# - 「誤変換を含む文（wrong_text）」をモデル入力とし、「正しい文（correct_text）」の
#   トークンID列を **教師ラベル（labels）** として与える最小例。
# - これにより各位置 t で 「正しいトークンID」を分類する **トークン分類問題** として学習できる。
#   * 推奨損失：CrossEntropyLoss（語彙サイズ |V| クラスの多クラス分類、PAD 等は ignore_index=-100 で除外）
# - 注意：max_length を超えると **切り詰め（truncation）** されるため、wrong と correct の整合が崩れる。
#   学習で両系列が等長にパディング・切り詰めされること（同じ max_length/手順）を必ず保証すること。
# - 出力の encoding には、wrong_text の BERT 入力（input_ids/attention_mask/token_type_ids）に加えて、
#   correct_text の input_ids が **encoding['labels']** として格納される実装（9-4 の SC_tokenizer 参照）。
# =============================================================================

wrong_text = "優勝トロフィーを変換した"  # 誤変換を含む入力（例：返還→変換）
correct_text = "優勝トロフィーを返還した"  # 正しい表記
encoding = tokenizer.encode_plus_tagged(wrong_text, correct_text, max_length=12)
print(encoding)
# 期待される中身（例）：
#  - 'input_ids'       : wrong_text をトークナイズ→ID化→[CLS]/[SEP] 付与→max_length へパディング/切り詰め
#  - 'token_type_ids'  : 単文なので通常は全 0
#  - 'attention_mask'  : 実トークン 1 / PAD 0
#  - 'labels'          : correct_text の input_ids（= 教師）。学習側で PAD/特殊トークンに -100 を適用推奨。
# 理論メモ：
#  - 本手法は「置換型誤り」には強い一方、挿入/削除や語順入れ替え等の編集距離を伴う誤りには弱い。
#    その場合は、アライメント（diff/LCS）で整列して等長化するか、seq2seq 系（エンコーダ–デコーダ）を検討する。

{'input_ids': [2, 759, 18204, 11, 4618, 15, 10, 3, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], 'labels': [2, 759, 18204, 11, 8274, 15, 10, 3, 0, 0, 0, 0]}


In [6]:
# 9-7
# =============================================================================
# 目的（理論）：
# - 学習/推論時に、原文テキストの各サブワード（WordPiece）が「元テキスト中のどの位置（span）」に
#   対応するかを取得する最小例である。これにより、トークン分類の予測結果（各位置の語彙ID）を
#   文字列へ正確に復号（置換）できる。
#
# 背景：
# - encode_plus_untagged は、形態素分割（MeCab）→サブワード分割（WordPiece）の後、
#   各サブワードから '##' を除いた素片を原文に前方探索でアラインし、[start, end) の半開区間で
#   スパン列（spans）を構築する。
# - prepare_for_model により [CLS]/[SEP] の付与とパディング/切り詰めが行われるため、
#   それら特殊トークンや PAD に対応する spans は [-1, -1] の番兵を入れておく（「実体なし」の意味）。
#
# 出力の読み方：
# - encoding: dict（'input_ids', 'token_type_ids', 'attention_mask' など）。return_tensors='pt' のため
#   各値は形状 [1, T] の torch.Tensor（バッチ次元 1 を含む）。
# - spans: 長さ T のリスト。各要素は [start, end]（原文の0始まりインデックス）。
#   * special/PAD の位置は [-1, -1]。
#   * 先頭は [CLS] のため必ず [-1, -1]、終端側も [SEP] と PAD による [-1, -1] が並ぶ。
#
# 注意点：
# - 前方貪欲一致は、同一部分文字列が繰り返される文や空白が多い文で誤マッチのリスクがある。
#   必要に応じて LCS/編集距離によるアライメントでロバスト化を検討。
# - 学習・推論で NFKC 正規化ポリシーを統一しないと、スパン境界と生成表記の差異が生じ得る。
# =============================================================================

wrong_text = "優勝トロフィーを変換した"
encoding, spans = tokenizer.encode_plus_untagged(wrong_text, return_tensors="pt")
print("# encoding")
print(encoding)
print("# spans")
print(spans)

# encoding
{'input_ids': tensor([[    2,   759, 18204,    11,  4618,    15,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
# spans
[[-1, -1], [0, 2], [2, 7], [7, 8], [8, 10], [10, 11], [11, 12], [-1, -1]]


In [7]:
# 9-8
# =============================================================================
# 目的（理論）：
# - 9-7 で得た spans（原文中のサブワード位置列）と、各位置の **予測トークンID列（predicted_labels）** を
#   用いて、SC_tokenizer.convert_bert_output_to_text により「正しい文章」を復元する最小例。
#
# 処理の流れ（convert_bert_output_to_text の要点）：
#  1) 事前条件：len(spans) == len(labels)（ここで labels=predicted_labels）。
#     - spans は [CLS], 本文サブワード, [SEP]（および PAD があれば PAD）に対応し、特殊/PAD は [-1, -1]。
#     - ラベル列も **同じ長さ**で、対応位置に語彙ID（token_id）を持つこと。
#  2) 特殊/PAD 位置の除去：
#     - span[0] == -1 の要素は「実体なし」として、labels/spans の両方から落とす。
#  3) サブワード復元：
#     - token_id → token へ変換し、WordPiece の接頭辞 '##' を除去して連結。
#     - 連結前に **NFKC 正規化**で全/半角・合成文字の揺れを矯正（学習/推論での正規化ポリシーは統一すること）。
#  4) 空白・非対象領域の保持：
#     - 連結中、原文の position と次のサブワード開始 start にギャップがあれば、原文の該当部分をそのまま複写。
#
# 理論メモ：
# - ここでの labels は「語彙サイズ |V| クラスの **トークン分類** の予測結果（argmax など）」を想定。
# - 位置合わせ（spans）は 9-7 の encode_plus_untagged で算出（MeCab→WordPiece→前方探索）。
#   反復文字列や空白が多い場合は前方貪欲一致が誤対応するリスクがあるため、大規模実運用では LCS/編集距離による
#   アライメントの頑健化を検討。
# - 予測列の長さが spans と一致しないと assert により失敗する。モデル出力 → ラベル化の段で
#   [CLS]/[SEP]/PAD を含んだ長さ合わせを行うこと。
# =============================================================================

predicted_labels = [
    2,
    759,
    18204,
    11,
    8274,
    15,
    10,
    3,
]  # 例：各位置の予測 token_id（語彙ID）
predicted_text = tokenizer.convert_bert_output_to_text(
    wrong_text, predicted_labels, spans
)
print(predicted_text)
# 出力想定：
# - モデルが '変換'→'返還' のような誤変換を修正できていれば、期待する正しい表記に復元される。
# - 語彙外/未知などで難しい場合は近傍のサブワード列に置換される可能性がある。

優勝トロフィーを返還した


In [9]:
# 9-9 / 9-10 の CUDA エラー修正（Mac/MPS・CPU でも動く汎用化）
# =============================================================================
# エラー原因：
# - 環境の PyTorch が「CUDA 非対応」でビルドされているため、`.cuda()` 呼び出しで失敗。
#   → Apple Silicon(Mac) は通常 CUDA が無いので、MPS または CPU を使う必要がある。
#
# 対応方針：
# 1) デバイスを自動選択（MPS → CUDA → CPU の順）。
# 2) モデル・テンソル移動は `.cuda()` ではなく **`.to(device)`** に統一。
# 3) 推論中心なら **`.eval()`** を付けてドロップアウト無効化。
# 4) 9-10 の `encoding` も `.to(device)` に変更。
# =============================================================================

import torch
from transformers import BertForMaskedLM

# --- 元のコード（参考：この2行が CUDA 非対応環境で落ちる） ---
# bert_mlm = BertForMaskedLM.from_pretrained(MODEL_NAME)
# bert_mlm = bert_mlm.cuda()


# --- 修正版：可搬なデバイス自動選択 ---
def pick_device():
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")  # Apple Silicon (Metal)
    if torch.cuda.is_available():
        return torch.device("cuda")  # NVIDIA CUDA
    return torch.device("cpu")  # フォールバック


device = pick_device()
bert_mlm = BertForMaskedLM.from_pretrained(MODEL_NAME).to(device).eval()
print(f"[info] bert_mlm loaded on: {device}")

# =============================================================================
# 9-10（修正版）：encoding も `.to(device)` に統一
#  - 注意：MLM は本来 [MASK] 位置のみを予測するタスク。全位置 argmax は自己再現に寄りやすい。
#    実運用では、怪しい部分を [MASK] 化して top-k から候補を選ぶ方が安定。
# =============================================================================

# 前段で用意済みの tokenizer / SC_tokenizer とする
text = "優勝トロフィーを変換した。"

# spans（原文中のサブワード位置）も同時取得
encoding, spans = tokenizer.encode_plus_untagged(text, return_tensors="pt")
encoding = {
    k: v.to(device) for k, v in encoding.items()
}  # ← ここを .cuda() ではなく .to(device)

with torch.no_grad():
    output = bert_mlm(**encoding)  # logits 形状：[B=1, T, |V|]
    scores = output.logits
    labels_predicted = scores[0].argmax(-1).to("cpu").numpy().tolist()

# 予測トークン列 → 文字列復元（special/PAD は内部で除外、'##' は除去、NFKC で正規化）
predict_text = tokenizer.convert_bert_output_to_text(text, labels_predicted, spans)

print("# input : ", text)
print("# output: ", predict_text)

# =============================================================================
# メモ（理論）：
# - 学習側（9-4）の encode_plus_tagged は「正解文の input_ids を labels に格納」→
#   各位置の多クラス CE 損失で学習できる（PAD/特殊は ignore_index=-100 を推奨）。
# - 推論側（本コード）は “全位置 argmax” のため、真の誤り訂正というより自己再現に寄る。
#   改善案：疑わしい位置のみ [MASK]、top-k 候補をビーム探索 or 言語的制約で選ぶ。
# =============================================================================

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[info] bert_mlm loaded on: mps
# input :  優勝トロフィーを変換した。
# output:  優勝トロフィーを獲得した。


In [12]:
# 9-11（説明コメント付き：MLM で「正解文トークン列」を教師にした最小学習ループ）
# =============================================================================
# 目的（理論）：
# - 誤変換文 wrong_text を入力 x とし、正しい文 correct_text の input_ids を教師 y（各位置の正解トークンID）として
#   BertForMaskedLM による **トークンごとの多クラス分類（語彙サイズ |V|）** で学習する最小例。
# - CrossEntropyLoss は「各位置 t のロジット z_{t,*} と教師 label_{t}（語彙ID）」で計算。
#   ただし **[CLS]/[SEP]/[PAD] など損失に含めたくない位置は -100（ignore_index）** にするのが定石。
# - 注意：MLM は本来 [MASK] 位置のみの復元で事前学習されているため、全位置を一様に監督すると
#   「自己再現」に寄りやすい。実務では（1）誤り疑い位置だけを [MASK] 化、（2）候補 top-k から置換判定、
#   などを併用すると安定。
# =============================================================================

data = [
    {
        "wrong_text": "優勝トロフィーを変換した。",
        "correct_text": "優勝トロフィーを返還した。",
    },
    {
        "wrong_text": "人と森は強制している。",
        "correct_text": "人と森は共生している。",
    },
]

# 各データを符号化し、データローダへ入力できる形に整形
# - encode_plus_tagged（9-4 で実装済み想定）は：
#     input_ids/attention_mask/token_type_ids（= wrong_text 側）に加え、
#     labels として correct_text の input_ids を格納する。
# - max_length は wrong/correct 双方で同じ長さになる（prepare_for_modelの挙動）。
max_length = 32
dataset_for_loader = []
for sample in data:
    wrong_text = sample["wrong_text"]
    correct_text = sample["correct_text"]
    encoding = tokenizer.encode_plus_tagged(
        wrong_text, correct_text, max_length=max_length
    )
    # Transformers の損失は long のラベル ID を想定（-100 も long）
    encoding = {k: torch.tensor(v) for k, v in encoding.items()}
    dataset_for_loader.append(encoding)

# データローダを作成（小サンプルなので 1 バッチ）
dataloader = DataLoader(dataset_for_loader, batch_size=2)

# --- デバイス選択：bert_mlm の実デバイスに合わせる（.cuda() は使わない） ---
device = next(bert_mlm.parameters()).device  # 例：mps / cuda / cpu

# トークナイザから特殊トークンIDを取得（BERT系では多くが [PAD]=0, [CLS]=101, [SEP]=102）
pad_id = tokenizer.pad_token_id
cls_id = tokenizer.cls_token_id
sep_id = tokenizer.sep_token_id

# ミニバッチを BERT へ入力し、損失を計算
# - ラベルをそのまま渡すと特殊/PAD まで学習対象になるため、ignore_index=-100 に置換する
bert_mlm.train()  # 学習想定（推論なら .eval() と no_grad を使う）
for batch in dataloader:
    # デバイスへ移動
    batch = {k: v.to(device) for k, v in batch.items()}

    # --- ignore_index マスクの構築（損失に含めない位置を -100 にする）---
    # 1) PAD 位置（attention_mask==0）を除外
    labels = batch["labels"].clone()
    amask = batch["attention_mask"]  # 1=実トークン, 0=PAD
    labels[amask == 0] = -100

    # 2) [CLS]/[SEP] も除外（多くのケースで学習対象外にするのが定石）
    inp = batch["input_ids"]
    labels[inp == cls_id] = -100
    labels[inp == sep_id] = -100

    # BertForMaskedLM は labels を与えると内部で CrossEntropyLoss を計算して output.loss を返す
    # （ignore_index は -100 が既定）
    # token_type_ids が無い場合は自動で None 扱いだが、あるなら一緒に渡す
    output = bert_mlm(
        input_ids=inp,
        attention_mask=amask,
        token_type_ids=batch.get("token_type_ids", None),
        labels=labels,
    )
    loss = output.loss  # 損失（バッチ×時系列の平均）

    # 例：逆伝播（最小例。実運用では optimizer/scheduler/AMP/勾配クリップ等を併用）
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()

    print(f"loss: {loss.item():.4f}")

# =============================================================================
# 追加メモ（理論と実装の橋渡し）：
# - 「正解文の input_ids を labels にする」設計は **トークン置換** には有効。
#   ただし挿入/削除や語順入れ替えが頻出する場合、位置合わせの仮定が崩れるため seq2seq/CTC などを検討。
# - 本設計を“MLMらしく”するなら、wrong_text のうち置換したい部分だけを [MASK] 化し、
#   labels も該当位置以外は -100 にして監督する手もある（局所最適化&副作用低減）。
# - 実学習では AdamW + weight decay、学習率 1e-5〜5e-5、warmup/linear スケジューラ、
#   勾配クリップ、mixed precision 等を併用すると安定。
# =============================================================================

loss: 0.7928


In [13]:
# 9-12
!curl -L "https://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=https://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JWTD/jwtd.tar.gz&name=JWTD.tar.gz" -o JWTD.tar.gz
!tar zxvf JWTD.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   244  100   244    0     0    168      0  0:00:01  0:00:01 --:--:--   168
100 64.9M  100 64.9M    0     0  1772k      0  0:00:37  0:00:37 --:--:-- 2082k
x jwtd/
x jwtd/train.jsonl
x jwtd/test.jsonl


In [14]:
# 9-4（説明コメント付き）：誤変換訂正タスク用データセット作成（トークン対応制約あり）
# -----------------------------------------------------------------------------
# 目的：
# - JWTD（日本語誤変換データ）から「漢字変換」に関するサンプルのみを抽出し、
#   BERT 推論/学習に適した形式（wrong_text, correct_text のペア配列）へ整形する。
# - SC_tokenizer（BertJapaneseTokenizer系）でトークン列を比較し、
#   「トークン数が一致」かつ「差分トークン数が閾値以下」のものだけを採用して
#   “位置合わせ” を単純化する（= token-level 監督が可能になる）。
#
# 理論メモ：
# - 本手法は “置換型誤り” を前提とするため、挿入/削除を伴うケースは除外する設計になっている。
#   （トークン数一致の制約）→ 位置 t ごとに語彙多クラス分類として監督できる。
# - 事前に NFKC 正規化（互換分解＋合成）を行い、全/半角や互換文字の揺れを吸収して
#   トークン化の安定性・比較の一貫性を高めている。
# - Pandas の `query(..., inplace=True)` / `rename(..., inplace=True)` は
#   引数 DataFrame を破壊的に更新する点に注意（呼び出し側とデータ再利用の設計に影響）。
# -----------------------------------------------------------------------------


def create_dataset(data_df):

    tokenizer = SC_tokenizer.from_pretrained(
        MODEL_NAME
    )  # SC_tokenizer は 9-4 で定義済み想定
    # BertJapaneseTokenizer ベースの tokenize() を使用

    def check_token_count(row):
        """
        誤変換の文章と正しい文章でトークンに対応がつくかどうかを判定。
        （条件は上の文章を参照）

        判定ロジックの要点：
        - WordPiece トークン列の長さが一致（= 挿入/削除なし）。
        - 差分トークン数 <= 閾値（“局所的な置換” のみを許容）。
          ※ 閾値（threthold_count）は実験設計上のハイパーパラメータ。
        """
        wrong_text_tokens = tokenizer.tokenize(row["wrong_text"])
        correct_text_tokens = tokenizer.tokenize(row["correct_text"])

        # 1) トークン数一致：位置合わせ（t 対応）を保証するための必須条件
        if len(wrong_text_tokens) != len(correct_text_tokens):
            return False

        # 2) 差分が“少数”であることを要求（局所置換の想定）
        diff_count = 0
        threthold_count = (
            2  # NOTE: 'threshold' の綴りミスだが、機能上は問題なし（修正は任意）
        )
        for wrong_text_token, correct_text_token in zip(
            wrong_text_tokens, correct_text_tokens
        ):

            if wrong_text_token != correct_text_token:
                diff_count += 1
                if diff_count > threthold_count:
                    # 許容差分を超えたら除外（挿入/削除に近い複雑な差分や広範な置換を排除）
                    return False
        return True

    def normalize(text):
        """
        文字列の正規化：
        - 前後空白の除去（strip）
        - NFKC 正規化（互換文字の正規化、全半角統一など）
        ※ 学習・推論・評価の全段で同じ正規化を適用することで、一貫性を担保するのが原則。
        """
        text = text.strip()
        text = unicodedata.normalize("NFKC", text)
        return text

    # --- データ抽出：カテゴリが「漢字誤変換」のものだけを残す（破壊的操作に注意） ---
    category_type = "kanji-conversion"
    data_df.query(
        "category == @category_type", inplace=True
    )  # inplace=True なので data_df が更新される
    data_df.rename(
        columns={"pre_text": "wrong_text", "post_text": "correct_text"}, inplace=True
    )

    # --- 正規化とフィルタリング ---
    # 1) 文字列正規化（NFKC）で表記揺れを抑制
    data_df["wrong_text"] = data_df["wrong_text"].map(normalize)
    data_df["correct_text"] = data_df["correct_text"].map(normalize)

    # 2) 統計出力のためにフィルタ前件数を保持
    kanji_conversion_num = len(data_df)

    # 3) トークン対応が成立するサンプルのみ抽出（挿入/削除や大規模置換を排除）
    data_df = data_df[data_df.apply(check_token_count, axis=1)]
    same_tokens_count_num = len(data_df)

    # 4) 抽出率のログ（学習データの “素性” を把握して再現性を高める）
    print(
        f"- 漢字誤変換の総数：{kanji_conversion_num}",
        f"- トークンの対応関係のつく文章の総数: {same_tokens_count_num}",
        f"  (全体の{same_tokens_count_num/kanji_conversion_num*100:.0f}%)",
        sep="\n",
    )

    # 返り値：学習・検証・テスト作成で共通利用しやすい、辞書リスト形式（records）
    # 例：{'wrong_text': '...', 'correct_text': '...'}
    return data_df[["wrong_text", "correct_text"]].to_dict(orient="records")


# --- データのロード（train/test の JSON Lines） ---
# 期待スキーマ（例）：
#   {
#     "category": "kanji-conversion",
#     "pre_text": "...",   # 誤変換文
#     "post_text": "..."   # 正しい文
#     ...
#   }
# ※ 列名は直後に rename 済み（wrong_text, correct_text）
train_df = pd.read_json("./jwtd/train.jsonl", orient="records", lines=True)
test_df = pd.read_json("./jwtd/test.jsonl", orient="records", lines=True)

# --- 学習＋検証用データセットの作成 ---
print("学習と検証用のデータセット：")
dataset = create_dataset(train_df)

# シャッフル（※ 乱数 seed 未固定。再現実験では random.seed の固定推奨）
random.shuffle(dataset)
n = len(dataset)

# ホールドアウト分割（8:2）。データ分布に偏りがある場合は層化分割の検討が望ましい。
n_train = int(n * 0.8)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:]

# --- テスト用データセットの作成 ---
print("テスト用のデータセット：")
dataset_test = create_dataset(test_df)

# 補足（設計上の注意）：
# - この段階では “置換型誤り” に限定したデータだけが残るため、
#   モデルが学習する分布も置換中心にバイアスされる。
#   実運用で挿入/削除・語順の乱れが多い場合、seq2seq/CTC など別定式化の導入を検討。
# - Pandas の inplace 操作により、呼び出し元の DataFrame が更新される点に注意。
#   他処理と共有するなら .copy() 運用を推奨（ここでは元コードを尊重して未変更）。

学習と検証用のデータセット：


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


- 漢字誤変換の総数：235490
- トークンの対応関係のつく文章の総数: 173992
  (全体の74%)
テスト用のデータセット：


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


- 漢字誤変換の総数：3061
- トークンの対応関係のつく文章の総数: 2289
  (全体の75%)
